# Linear Regression 

In [1]:
import warnings

warnings.filterwarnings('ignore')

import pandas as pd 
import numpy as np

from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin


from mypipes_linear import *

In [2]:
train_file=r'/home/siddarth.jha@npci.org.in/Documents/Training/Data/loan_data_train.csv'
test_file=r'/home/siddarth.jha@npci.org.in/Documents/Training/Data/loan_data_test.csv'

ld_train=pd.read_csv(train_file)
ld_test=pd.read_csv(test_file)               


In [3]:
ld_train.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years
2,67265.0,2100,2100,14.33%,36 months,major_purchase,3.50%,LA,OWN,1000.00,690-694,13,893,1.0,< 1 year
3,80167.0,28000,28000,16.29%,36 months,credit_card,19.62%,NV,MORTGAGE,7083.33,710-714,12,38194,1.0,10+ years
4,17240.0,24250,17431.82,12.23%,60 months,credit_card,23.79%,OH,MORTGAGE,5833.33,730-734,6,31061,2.0,10+ years


In [4]:
ld_test.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,20093,5000,5000,60 months,moving,12.59%,NY,RENT,4416.67,690-694,13,7686,0,< 1 year
1,62445,18000,18000,60 months,debt_consolidation,4.93%,CA,RENT,5258.50,710-714,6,11596,0,10+ years
2,65248,7200,7200,60 months,debt_consolidation,25.16%,LA,MORTGAGE,3750.00,750-754,13,7283,0,6 years
3,81822,7200,7200,36 months,debt_consolidation,17.27%,NY,MORTGAGE,3416.67,790-794,14,4838,0,10+ years
4,57923,22000,22000,60 months,debt_consolidation,18.28%,MI,MORTGAGE,6083.33,720-724,9,20181,0,8 years


In [5]:
ld_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              2199 non-null   float64
 1   Amount.Requested                2199 non-null   object 
 2   Amount.Funded.By.Investors      2199 non-null   object 
 3   Interest.Rate                   2200 non-null   object 
 4   Loan.Length                     2199 non-null   object 
 5   Loan.Purpose                    2199 non-null   object 
 6   Debt.To.Income.Ratio            2199 non-null   object 
 7   State                           2199 non-null   object 
 8   Home.Ownership                  2199 non-null   object 
 9   Monthly.Income                  2197 non-null   float64
 10  FICO.Range                      2200 non-null   object 
 11  Open.CREDIT.Lines               2196 non-null   object 
 12  Revolving.CREDIT.Balance        21

In [6]:
ld_train.sample(10)

# drop columns : Amount.Funded.By.Investors , ID, Interest.Rate

#1 Amount requested : convert it to numeric

#2 Loan Length : create dummies with frequency cutoff 20

#3 Loan.Purpose : dummies with freq cutoff

#4  Debt.To.Income.Ratio : remove % and then convert to numeric

#5  State: dummies with frequency cutoff

#6 Home.Ownership : dummies with frequency cutoff

#7 Monthly Income : as is

#8 FICO.Range : break a-b , in to a ,b , convert them to numeric 
# then create new column fico=0.5*(a+b) and then drop the original FICO.Range

#9 Open Credit Lines : convert to numeric

#10 Revolving Credit balance : convert it to numeric 

#11 inquiries in the last 6 months : as is

#12 Employment.Length : create dummies with frequency cutoff

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
515,53780.0,12500,12475,6.03%,36 months,car,14.82%,FL,RENT,4944.92,755-759,11,6014,0.0,2 years
597,46870.0,14600,14600,9.76%,36 months,credit_card,29.51%,FL,OWN,2500.00,730-734,14,23616,1.0,2 years
1398,76797.0,32550,32550,6.62%,36 months,debt_consolidation,8.52%,IL,MORTGAGE,9166.67,805-809,9,2670,0.0,10+ years
13,47011.0,3600,3600,13.67%,36 months,other,16.41%,CA,MORTGAGE,5113.00,690-694,10,715,1.0,NaN
206,33155.0,14000,14000,16.77%,36 months,credit_card,9.67%,FL,MORTGAGE,12083.33,670-674,13,28318,0.0,< 1 year
754,23644.0,2700,2700,5.79%,36 months,debt_consolidation,19.81%,OH,MORTGAGE,3958.33,750-754,14,13159,1.0,8 years
348,46097.0,6400,6400,10.74%,36 months,debt_consolidation,19.23%,DC,RENT,3833.33,690-694,13,10451,0.0,< 1 year
720,82160.0,9500,9500,12.12%,36 months,debt_consolidation,24.06%,MO,MORTGAGE,2519.17,705-709,7,8673,1.0,10+ years
110,16667.0,3000,.,17.44%,36 months,other,15.79%,MD,RENT,7952.00,660-664,4,12578,0.0,8 years
1336,18161.0,8000,8000,14.09%,36 months,home_improvement,7.68%,IL,RENT,25000.00,675-679,6,28579,1.0,8 years


In [7]:
p1=pdPipeline([
    ('var_select',VarSelector(['Amount.Requested','Open.CREDIT.Lines','Revolving.CREDIT.Balance'])),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

p2=pdPipeline([
    ('var_select',VarSelector(['Debt.To.Income.Ratio'])),
    ('string_clean',string_clean(replace_it='%',replace_with='')),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

p3=pdPipeline([
    ('var_select',VarSelector(['Loan.Length', 'Loan.Purpose','State','Home.Ownership'])),
    ('missing_trt',DataFrameImputer()),
    ('create_dummies',get_dummies_Pipe(20))
])

p4=pdPipeline([
    ('var_select',VarSelector(['Monthly.Income','Inquiries.in.the.Last.6.Months'])),
    ('missing_trt',DataFrameImputer())
])

p5=pdPipeline([
    ('var_select',VarSelector(['FICO.Range'])),
    ('custom_fico',custom_fico()),
    ('missing_trt',DataFrameImputer())
])

p6 = pdPipeline([
    ('var_select', VarSelector(['Employment.Length'])),
    ('custom_El', custom_El()),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt', DataFrameImputer())
])

data_pipe=FeatureUnion([
    ('obj_to_num',p1),
    ('dtir',p2),
    ('obj_to_dum',p3),
    ('num',p4),
    ('fico',p5),
    ('el', p6)
])

In [8]:
x_train=pd.DataFrame(data=data_pipe.fit_transform(ld_train),
                     columns=data_pipe.get_feature_names())

In [9]:
x_train.shape

(2200, 49)

In [10]:
x_train

,obj_to_num__Amount.Requested,obj_to_num__Open.CREDIT.Lines,obj_to_num__Revolving.CREDIT.Balance,dtir__Debt.To.Income.Ratio,obj_to_dum__Loan.Length_36 months,obj_to_dum__Loan.Length_60 months,obj_to_dum__Loan.Purpose_debt_consolidation,obj_to_dum__Loan.Purpose_credit_card,obj_to_dum__Loan.Purpose_other,obj_to_dum__Loan.Purpose_home_improvement,...,obj_to_dum__State_WI,obj_to_dum__State_OR,obj_to_dum__State_KS,obj_to_dum__Home.Ownership_MORTGAGE,obj_to_dum__Home.Ownership_RENT,obj_to_dum__Home.Ownership_OWN,num__Monthly.Income,num__Inquiries.in.the.Last.6.Months,fico__fico,el__El
0,25000.0,11.0,15210.0,27.56,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,8606.56,3.0,722.0,5.0
1,19750.0,14.0,19070.0,13.39,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,6737.50,3.0,712.0,4.0
2,2100.0,13.0,893.0,3.50,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1000.00,1.0,692.0,0.0
3,28000.0,12.0,38194.0,19.62,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,7083.33,1.0,712.0,10.0
4,24250.0,6.0,31061.0,23.79,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,5833.33,2.0,732.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,30000.0,16.0,17969.0,12.10,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,7083.33,1.0,677.0,10.0
2196,4200.0,4.0,5718.0,14.16,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,3850.00,0.0,702.0,2.0
2197,19800.0,10.0,46879.0,15.03,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,6666.67,3.0,677.0,6.0
2198,18000.0,5.0,32394.0,11.63,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,9051.83,2.0,672.0,4.0


In [11]:
x_test=pd.DataFrame(data=data_pipe.transform(ld_test),
                     columns=data_pipe.get_feature_names())

In [12]:
x_test.shape

(300, 49)

In [13]:
x_train.head()

,obj_to_num__Amount.Requested,obj_to_num__Open.CREDIT.Lines,obj_to_num__Revolving.CREDIT.Balance,dtir__Debt.To.Income.Ratio,obj_to_dum__Loan.Length_36 months,obj_to_dum__Loan.Length_60 months,obj_to_dum__Loan.Purpose_debt_consolidation,obj_to_dum__Loan.Purpose_credit_card,obj_to_dum__Loan.Purpose_other,obj_to_dum__Loan.Purpose_home_improvement,...,obj_to_dum__State_WI,obj_to_dum__State_OR,obj_to_dum__State_KS,obj_to_dum__Home.Ownership_MORTGAGE,obj_to_dum__Home.Ownership_RENT,obj_to_dum__Home.Ownership_OWN,num__Monthly.Income,num__Inquiries.in.the.Last.6.Months,fico__fico,el__El
0,25000.0,11.0,15210.0,27.56,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,8606.56,3.0,722.0,5.0
1,19750.0,14.0,19070.0,13.39,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,6737.50,3.0,712.0,4.0
2,2100.0,13.0,893.0,3.50,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1000.00,1.0,692.0,0.0
3,28000.0,12.0,38194.0,19.62,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,7083.33,1.0,712.0,10.0
4,24250.0,6.0,31061.0,23.79,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,5833.33,2.0,732.0,10.0


In [14]:
x_test.head()

,obj_to_num__Amount.Requested,obj_to_num__Open.CREDIT.Lines,obj_to_num__Revolving.CREDIT.Balance,dtir__Debt.To.Income.Ratio,obj_to_dum__Loan.Length_36 months,obj_to_dum__Loan.Length_60 months,obj_to_dum__Loan.Purpose_debt_consolidation,obj_to_dum__Loan.Purpose_credit_card,obj_to_dum__Loan.Purpose_other,obj_to_dum__Loan.Purpose_home_improvement,...,obj_to_dum__State_WI,obj_to_dum__State_OR,obj_to_dum__State_KS,obj_to_dum__Home.Ownership_MORTGAGE,obj_to_dum__Home.Ownership_RENT,obj_to_dum__Home.Ownership_OWN,num__Monthly.Income,num__Inquiries.in.the.Last.6.Months,fico__fico,el__El
0,5000.0,13.0,7686.0,12.59,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,4416.67,0.0,692.0,0.0
1,18000.0,6.0,11596.0,4.93,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,5258.50,0.0,712.0,10.0
2,7200.0,13.0,7283.0,25.16,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,3750.00,0.0,752.0,6.0
3,7200.0,14.0,4838.0,17.27,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,3416.67,0.0,792.0,10.0
4,22000.0,9.0,20181.0,18.28,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,6083.33,0.0,722.0,8.0


In [15]:
y_train=ld_train['Interest.Rate'].str.replace('%','').astype(float)
y_train

0       18.49
1       17.27
2       14.33
3       16.29
4       12.23
        ...  
2195    23.28
2196    14.33
2197    15.31
2198    20.99
2199    10.16
Name: Interest.Rate, Length: 2200, dtype: float64

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [17]:
lm=LinearRegression()

In [18]:
lm.fit?

In [19]:
cv_mae=-cross_val_score(lm,
                        x_train,y_train,
                        cv=10,
                        scoring='neg_mean_absolute_error')
# The scoring ending with _error -> The lower the better. Returns a value to minimize.
# The scoring ending with _score returns the value to maximize. The higher the better.

In [20]:
cv_mae
# array([1.72180364, 1.77190797, 1.76961603, 1.64664953, 1.42368696,
#        1.64323402, 1.42325966, 1.58390193, 1.52837312, 1.65977978])

array([1.72180364, 1.77190797, 1.76961603, 1.64664953, 1.42368696,
       1.64323402, 1.42325966, 1.58390193, 1.52837312, 1.65977978])

In [21]:
cv_mae.mean()
# 1.6172212632192573

1.6172212632192573

In [22]:
cv_mae.std()
# 0.12063346581841543

0.12063346581841543

In [23]:
lm.fit(x_train,y_train)

LinearRegression()

In [24]:
lm.intercept_

73.47738349332741

In [25]:
lm.coef_

array([ 1.60452569e-04, -3.67626112e-02, -2.95814271e-06, -1.13478975e-04,
        1.28299898e+00,  4.44954062e+00, -7.61550279e-01, -8.77320686e-01,
        5.39014167e-02, -6.78853188e-01, -4.05500939e-01, -2.66839810e-01,
       -5.13444172e-01, -9.88244463e-01, -5.91702806e-01,  7.36391892e-01,
       -1.84408382e-01, -1.89757625e-01, -5.38834548e-02,  4.54438848e-01,
       -4.83141700e-01, -5.82163968e-01, -2.17333017e-01, -3.82711456e-01,
       -1.19387332e-01, -8.76851289e-02, -4.91988156e-01, -2.32449904e-01,
       -1.01185661e-01,  3.93822172e-02, -2.68097716e-01,  3.94510930e-01,
        4.04481744e-01, -8.45116666e-02, -8.78376500e-02, -1.85210283e-01,
       -3.57879036e-01, -2.33991666e-01, -4.68447519e-02,  3.41456756e-02,
        4.36148637e-02,  2.01591311e-01, -2.37914879e+00, -2.16167598e+00,
       -2.04384661e+00, -2.59827967e-05,  3.38382544e-01, -8.64781118e-02,
        1.98050528e-02])

In [26]:
list(zip(x_train.columns,lm.coef_))

[('obj_to_num__Amount.Requested', 0.00016045256883493348),
 ('obj_to_num__Open.CREDIT.Lines', -0.036762611160864475),
 ('obj_to_num__Revolving.CREDIT.Balance', -2.9581427083155503e-06),
 ('dtir__Debt.To.Income.Ratio', -0.00011347897493044212),
 ('obj_to_dum__Loan.Length_36 months', 1.2829989784089877),
 ('obj_to_dum__Loan.Length_60 months', 4.449540624009991),
 ('obj_to_dum__Loan.Purpose_debt_consolidation', -0.7615502794175302),
 ('obj_to_dum__Loan.Purpose_credit_card', -0.87732068628948),
 ('obj_to_dum__Loan.Purpose_other', 0.05390141671472698),
 ('obj_to_dum__Loan.Purpose_home_improvement', -0.6788531881770699),
 ('obj_to_dum__Loan.Purpose_major_purchase', -0.40550093901849227),
 ('obj_to_dum__Loan.Purpose_small_business', -0.26683980972117727),
 ('obj_to_dum__Loan.Purpose_car', -0.513444172367349),
 ('obj_to_dum__Loan.Purpose_wedding', -0.9882444628027013),
 ('obj_to_dum__Loan.Purpose_medical', -0.5917028055548262),
 ('obj_to_dum__Loan.Purpose_moving', 0.736391892139872),
 ('obj_to

In [27]:
test_pred=lm.predict(x_test)

In [28]:
test_pred

array([16.65445553, 15.94100238, 10.42902509,  3.6728478 , 15.40221551,
        6.98876239, 15.77482829, 10.62384888, 15.98556653, 12.38422695,
        9.68353254, 14.90631287, 11.34752622, 13.70119681, 13.47443272,
       18.39020713, 10.38296505, 15.74398021, 13.76672916, 13.93990006,
       22.44485299, 17.4515836 , 12.13725328, 14.83120041,  9.68386727,
       11.17199872, 13.10447846, 19.43435575, 12.00352632, 17.0156246 ,
       15.27883728, 15.46646287, 12.37477237, 15.21379981, 13.72365174,
       14.12789549, 19.49154375, 11.25181443, 12.01130698, 16.98817784,
       14.14051537, 11.07691669, 14.64074014, 13.11537395, 15.59014435,
       17.02379181, 14.84241809, 19.04412818, 16.92854421, 10.4187541 ,
       13.6037736 , 19.79439364, 10.02634109, 19.66532807, 15.61976754,
       14.88090767, 17.52183807, 14.33296689, 10.90605535, 14.43437201,
       13.11804993, 17.00358825,  8.81105062, 14.54018696, 10.55010137,
       11.46495771, 12.83505385, 15.08491855, 12.07238449, 14.49

We can write these to a csv file for submission like this :

In [29]:
pd.DataFrame(test_pred).to_csv("mysubmission.csv",index=False)

In [30]:
import os
os.getcwd()

'/home/siddarth.jha@npci.org.in/Documents/Training'

# Ridge  Regression

In [31]:
from sklearn.linear_model import Ridge,Lasso

from sklearn.model_selection import GridSearchCV



In [32]:
lambdas=np.linspace(1,100,100)

In [33]:
lambdas

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])

In [34]:
params={'alpha':lambdas}

In [35]:
model=Ridge(fit_intercept=True)

In [36]:
Ridge?

In [37]:
grid_search=GridSearchCV(model,
                         param_grid=params,
                         cv=10,
                         scoring='neg_mean_absolute_error',
                        verbose=20,n_jobs=-1)

In [38]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


KeyboardInterrupt: 

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.cv_results_

 if you want you can now fit a ridge regression model with obtained value of alpha , although there is no need, grid search automatically fits the best estimator on the entire data, you can directly use this to make predictions on test_data. But if you want to look at coefficients , its much more convenient to fit the model with direct function

Using the report function given below you can see the cv performance of top few models as well, that will the tentative performance

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.6f} (std: {1:.6f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
report(grid_search.cv_results_,5)

In [ ]:
test_pred=grid_search.predict(x_test)

In [ ]:
pd.DataFrame(test_pred).to_csv("mysubmission.csv",index=False)

## For looking at coefficients

In [ ]:
grid_search.best_estimator_

In [ ]:
ridge_model=grid_search.best_estimator_

In [ ]:
ridge_model.fit(x_train,y_train)

In [ ]:
list(zip(data_pipe.get_feature_names(),ridge_model.coef_))

In [ ]:
lm.coef_/ridge_model.coef_


## Lasso Regression

In [ ]:
lambdas=np.linspace(1,10,100)

model=Lasso()

params={'alpha':lambdas}

In [ ]:
grid_search=GridSearchCV(model,
                         param_grid=params,
                         cv=10,
                         scoring='neg_mean_absolute_error',
                        verbose=20,n_jobs=-1)

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_params_

In [ ]:
report(grid_search.cv_results_,5)

you can see that, the best value of alpha comes at the edge of the range that we tried , we should expand the trial range on that side and run this again

In [ ]:
lambdas=np.linspace(.001,2,100)

params={'alpha':lambdas}

In [ ]:
grid_search=GridSearchCV(model,param_grid=params,cv=10,scoring='neg_mean_absolute_error')
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
report(grid_search.cv_results_,5)

In [ ]:
lasso_model=grid_search.best_estimator_

In [ ]:
lasso_model.fit(x_train,y_train)

In [ ]:
lasso_model.intercept_

In [ ]:
list(zip(data_pipe.get_feature_names(),lasso_model.coef_))


In [ ]:
(lasso_model.coef_==0).sum()


# Logistic Regression

In [ ]:
import warnings

warnings.filterwarnings('ignore')

import pandas as pd 
import numpy as np

from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from mypipes_linear import *

In [ ]:
train_file=r'/home/siddarth.jha@npci.org.in/Documents/Training/Data/rg_train.csv'
test_file=r'/home/siddarth.jha@npci.org.in/Documents/Training/Data/rg_test.csv'
bd_train=pd.read_csv(train_file)

bd_test=pd.read_csv(test_file)

In [ ]:
bd_train.head()

In [ ]:
bd_train['age_band'].value_counts(dropna=False)

In [ ]:
# drop : REF_NO, post_area , post_code,Revenue.Grid 
# children : convert zero:0 and 4+: 4 and then convert to numeric 
# age_band : 71+ : 71, Unknown: NA, rest: split and average
# status, occupation, occupation_partner,home_status: create dummies with freq cutoff
# family_income : remove [,>=], 35000:35000, 4000: 4000, unknown:NA, rest : split then avg
# self_employed, self_employed_partner : dummies 
# year_last_moved : keep as is 
# TVarea : dummies 
# 'Average.Credit.Card.Transaction', 'Balance.Transfer',
#       'Term.Deposit', 'Life.Insurance', 'Medical.Insurance',
#       'Average.A.C.Balance', 'Personal.Loan', 'Investment.in.Mutual.Fund',
#       'Investment.Tax.Saving.Bond', 'Home.Loan', 'Online.Purchase.Amount'
# 'Investment.in.Commudity',
#       'Investment.in.Equity', 'Investment.in.Derivative',
#      'Portfolio.Balance' : as is 
# gender , region : dummies 
    

In [ ]:
bd_train.info()

In [ ]:
num_vars=list(bd_train.select_dtypes(exclude=['object']).columns)

In [ ]:
num_vars

In [ ]:
num_vars=[_ for _ in num_vars if _ not in ['REF_NO','Revenue.Grid']]

In [ ]:
num_vars

In [ ]:
cat_vars=list(bd_train.select_dtypes(include=['object']).columns)

In [ ]:
cat_vars

In [ ]:
cat_vars=[_ for _ in cat_vars if _ not in 
          ['children','age_band', 'post_code','post_area','family_income']]

In [ ]:
cat_vars

In [ ]:
p1=pdPipeline([
    ('var_select',VarSelector(num_vars)),
    ('missing_trt',DataFrameImputer())
])

In [ ]:
p2=pdPipeline([
    ('var_select',VarSelector(cat_vars)),
    ('missing_trt',DataFrameImputer()),
    ('create_dummies',get_dummies_Pipe(70))
])

In [ ]:
p3=pdPipeline([
    ('var_select',VarSelector(['age_band'])),
    ('custom_fico',custom_age_band()),
    ('missing_trt',DataFrameImputer())
])

p4=pdPipeline([
    ('var_select',VarSelector(['family_income'])),
    ('custom_fico',custom_family_income()),
    ('missing_trt',DataFrameImputer())
])

p5=pdPipeline([
    ('var_select',VarSelector(['children'])),
    ('string_clean1',string_clean(replace_it='Zero',replace_with='0')),
    ('string_clean2',string_clean(replace_it='4+',replace_with='4')),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

In [ ]:
data_pipe=FeatureUnion([
    ('num',p1),
    ('obj_to_dum',p2),
    ('age_band',p3),
    ('family_income',p4),
    ('children',p5)
])

In [ ]:
x_train=pd.DataFrame(data=data_pipe.fit_transform(bd_train),
                     columns=data_pipe.get_feature_names())


In [ ]:
x_test=pd.DataFrame(data=data_pipe.transform(bd_test),
                     columns=data_pipe.get_feature_names())

In [ ]:
bd_train['Revenue.Grid'].value_counts(dropna=False)

In [ ]:
y_train=(bd_train['Revenue.Grid']==1).astype(int)

In [ ]:
y_train.value_counts()

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
x_train.head()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LogisticRegression?

In [ ]:
params={'class_weight':['balanced',None],
        'penalty':['l1','l2'],
        'C':[.0001,.0005,.001,.005, .000001]}

In [ ]:
model=LogisticRegression(fit_intercept=True, solver='liblinear')

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_search=GridSearchCV(model,
                         param_grid=params,
                         cv=10,
                         scoring="roc_auc",
                         n_jobs=-1,
                         verbose=20)

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler

In [ ]:
# oversample = RandomUnderSampler(sampling_strategy=0.5)
# x_sample, y_sample = oversample.fit_resample(x_train, y_train)

In [ ]:
# y_sample.value_counts()

In [ ]:
grid_search.fit(x_train,y_train)
# GridSearchCV(cv=10, estimator=LogisticRegression(solver='liblinear'), n_jobs=-1,
#              param_grid={'C': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 1,
#                                2, 5],
#                          'class_weight': ['balanced', None],
#                          'penalty': ['l1', 'l2']},
#              scoring='roc_auc', verbose=20)

In [ ]:
type(y_train)

In [ ]:
grid_search.best_estimator_
# LogisticRegression(C=0.005, class_weight='balanced', penalty='l1',
#                    solver='liblinear')

In [ ]:
logr=grid_search.best_estimator_

In [ ]:
report(grid_search.cv_results_,5)

# Model with rank: 1
# Mean validation score: 0.954793 (std: 0.012707)
# Parameters: {'C': 0.005, 'class_weight': 'balanced', 'penalty': 'l1'}

# Model with rank: 2
# Mean validation score: 0.954763 (std: 0.012756)
# Parameters: {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l1'}

# Model with rank: 3
# Mean validation score: 0.954739 (std: 0.012947)
# Parameters: {'C': 0.05, 'class_weight': 'balanced', 'penalty': 'l1'}

# Model with rank: 4
# Mean validation score: 0.954738 (std: 0.012918)
# Parameters: {'C': 0.005, 'class_weight': 'balanced', 'penalty': 'l2'}

# Model with rank: 5
# Mean validation score: 0.954731 (std: 0.012832)
# Parameters: {'C': 0.0005, 'class_weight': 'balanced', 'penalty': 'l2'}


In [ ]:
LogisticRegression?

In [ ]:
logr=LogisticRegression(fit_intercept=True,
                        **{'C': 0.005, 'class_weight': 'balanced', 'penalty': 'l1'},solver='liblinear')
# default solver lbfgs does not support l1 penalty for some versions of sklearn
# if you get an error like that , simply use solver='liblinear', it supports both l1 & l2 penalty


In [ ]:
logr.fit(x_train,y_train)

In [ ]:
(logr.coef_[0]==0).sum()

In [ ]:
list(zip(x_train.columns,logr.coef_[0]))

In [ ]:
logr.predict_proba(x_test)

In [ ]:
logr.classes_

In [ ]:
cutoffs=np.linspace(0.01,0.99,99)

cutoffs

In [ ]:
logr.predict_proba(x_train)

In [ ]:
logr.classes_

In [ ]:
train_score=logr.predict_proba(x_train)[:,1]
real=y_train
 # In  order to find the probability of which column is for outcome 1 and which for outcome 0


In [ ]:
print(train_score.shape)
print(real.value_counts())

In [ ]:
(train_score>0.2).astype(int)

In [ ]:
KS_all=[]

for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)

    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    
    P=TP+FN
    N=TN+FP
    
      
    KS=(TP/P)-(FP/N)
       
    KS_all.append(KS)
    


In [ ]:
KS_all

In [ ]:
list(zip(cutoffs,KS_all))

In [ ]:
mycutoff=cutoffs[KS_all==max(KS_all)]
mycutoff

In [ ]:
logr.intercept_

In [ ]:
list(zip(x_train.columns,logr.coef_[0]))

if you simply had to submit probability scores , you could do this 

In [ ]:
logr.predict_proba(x_test)

In [ ]:
test_score=logr.predict_proba(x_test)[:,1]
test_score

In [ ]:
(test_score>mycutoff).astype(int)

In [ ]:
pd.DataFrame(test_score).to_csv("mysubmission.csv",index=False)

if you had to submit hardclasses , you can apply the cutoff obtained above and then submit

In [ ]:
test_classes=(test_score>mycutoff).astype(int)

In [ ]:
pd.DataFrame(test_classes).to_csv("mysubmission.csv",index=False)